In [1]:
!pip install optuna catboost

     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 76.6 MB 46 kB/s 
     |████████████████████████████████| 210 kB 36.7 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 78 kB 5.6 MB/s 
     |████████████████████████████████| 150 kB 34.9 MB/s 
     |████████████████████████████████| 113 kB 39.3 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=261317da1e315f429bcfe91ce0e27e4d94e9687874a377f9a6d11fd1b7737bd1
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import warnings
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline 

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.utils import resample
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
import optuna

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
train=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_train.csv")
val=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_val.csv")
test=pd.read_csv(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/trainning/funix_test.csv")

In [10]:
MERCHANT_COLUMNS_PROPERTIES=pd.read_excel(r"/content/drive/MyDrive/BIDV/CARD_FRAUD/data/column_description.xlsx")
def get_list_cat_columns():
    feature_cad = MERCHANT_COLUMNS_PROPERTIES.copy()
    feature_cad = feature_cad[(feature_cad["ORG_SOURCE"].isin(["CAD", "MIS_MRCH_DIM", "MIS_MR_CST_DIM","MIS_CST_FCT","MIS_MRCH_FCT","EXT"]))& (feature_cad["COLUMN_TP"] == "CAT")
    ]
    return feature_cad["COLUMN_NAME"].tolist()

In [11]:
def reduce_memory_usage(df):   
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                    
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [12]:

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [13]:
train = reduce_memory_usage(train)
val = reduce_memory_usage(val)
test = reduce_memory_usage(test)

Memory usage of dataframe is 1.166229248046875 MB
Memory usage of dataframe after reduction 0.3381690979003906 MB
Reduced by 71.00320554755986 % 
Memory usage of dataframe is 0.2066650390625 MB
Memory usage of dataframe after reduction 0.058624267578125 MB
Reduced by 71.63319551092735 % 
Memory usage of dataframe is 0.242523193359375 MB
Memory usage of dataframe after reduction 0.06910324096679688 MB
Reduced by 71.50654334969171 % 


In [14]:
train.head()

,ID,MER_TYPE,BRANCH,CURR_NUM,DAY_REINSTATED,DAYS_AVGE,FL_LIM1,FL_LIM3,FL_LIM4,GROSS_SALE,...,DAY_CLOSE_YEAR,DAY_CLOSE_MONTH,DAY_CLOSE_DAY,DAY_START_YEAR,DAY_START_MONTH,DAY_START_DAY,DAY_SUBM_YEAR,DAY_SUBM_MONTH,DAY_SUBM_DAY,MERCH_FR
0,2208,5411,126,704,0,1,150,0,5000,671981120.0,...,2099,12,31,1970,1,1,1970,1,1,1
1,53,7011,753,704,0,2,150,0,0,0.0,...,2016,11,14,1970,1,1,1970,1,1,0
2,52,5411,128,704,0,2,150,0,5000,204670160.0,...,2099,12,31,1970,1,1,1970,1,1,0
3,163,5977,741,704,0,3,150,0,0,0.0,...,2016,10,3,1970,1,1,1970,1,1,0
4,657,5411,721,704,0,1,150,0,5000,0.0,...,2020,7,24,1970,1,1,1970,1,1,0


In [15]:
useful_features = [c for c in train.columns if ((c not in ['ID', 'MERCH_FR', 'BRANCH']) and ("DAY_" not in c))] 
# list_cat_org = get_list_cat_columns() 
# object_cols = [c for c in useful_features if c in list_cat_org]
object_cols = ['CURR_NUM', 'MP_IND', 'PAY_METHOD', 'HAS_TXN_LESS_15S']
num_cols = [c for c in useful_features if c not in object_cols]
# df_test = val[useful_features]

In [16]:
x_train = train[useful_features]
x_val = val[useful_features]
x_test = test[useful_features]
y_train = train["MERCH_FR"]
y_val = val["MERCH_FR"]
y_test = test["MERCH_FR"]

In [17]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler, OneHotEncoder
from imblearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler() # ('scaler', MinMaxScaler()    
    )])

# cat_label_cols = ['CST_PERF_ST', 'CST_MKT_SEG']
cat_ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OrdinalEncoder() 
    )])

# cat_onehot_cols = [f for f in cat_cols if f not in cat_label_cols]
cat_onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore', sparse=False) 
    )])

# over sampling 
oversample = SMOTE() # ADASYN()
preprocessor = ColumnTransformer(
    transformers=[
                  ('num', numeric_transformer, num_cols), 
                  #('cat_label', cat_ordinal_transformer, cat_label_cols)
                  #('cat_onehot', cat_onehot_transformer, ['CLOSE_RSN','CHIP_YN']) # ['CLOSE_RSN', 'COMM_FREQ']
                 ], remainder='drop')

In [18]:
# preprocessor.fit(train[useful_features])
preprocessor.fit(x_train)

x_train_tf = pd.DataFrame(preprocessor.transform(x_train), columns = get_feature_names(preprocessor)) 
x_val_tf = pd.DataFrame(preprocessor.transform(x_val), columns = get_feature_names(preprocessor)) 
x_test_tf = pd.DataFrame(preprocessor.transform(x_test), columns = get_feature_names(preprocessor)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available


In [19]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(x_train_tf, y_train)
preds_val = grid.best_estimator_.predict(x_val_tf)
score = f1_score(y_val, preds_val)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.684 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.692 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.702 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.689 total time=   0.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.859 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.905 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.843 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.880 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.858 total time=   0.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.917 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [20]:
score

0.8601036269430052

In [21]:
grid.best_params_

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [22]:
best_params = grid.best_params_
model = SVC(        
        **best_params
    )
pipeline = Pipeline(steps=[('sampling', oversample),
                    ('classifier', model)])

pipeline.fit(x_train_tf, y_train)
preds_test = model.predict(x_test_tf)
f1score = f1_score(y_test, preds_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_test))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       225
           1       0.93      0.91      0.92       113

    accuracy                           0.95       338
   macro avg       0.94      0.94      0.94       338
weighted avg       0.95      0.95      0.95       338



# Pipeline 2, handle category data

In [23]:
# cat_label_cols = ['CST_PERF_ST', 'CST_MKT_SEG']
cat_ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('encoder', OrdinalEncoder() 
    )])

# cat_onehot_cols = [f for f in cat_cols if f not in cat_label_cols]
cat_onehot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_val')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore', sparse=False) 
    )])

preprocessor = ColumnTransformer(
    transformers=[
                  ('num', numeric_transformer, num_cols), 
                  ('cat_label', cat_ordinal_transformer, object_cols)
                  #('cat_onehot', cat_onehot_transformer, ['CLOSE_RSN','CHIP_YN']) # ['CLOSE_RSN', 'COMM_FREQ']
                 ], remainder='drop')

In [24]:
preprocessor.fit(x_train)

x_train_tf = pd.DataFrame(preprocessor.transform(x_train), columns = get_feature_names(preprocessor)) 
x_val_tf = pd.DataFrame(preprocessor.transform(x_val), columns = get_feature_names(preprocessor)) 
x_test_tf = pd.DataFrame(preprocessor.transform(x_test), columns = get_feature_names(preprocessor)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer cat_label (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer cat_label (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Transformer num (type Pipeline) does not provide get_feature_names. Will return input column names if available
/usr/local/lib/python3.7/dist-packages

In [26]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# fitting the model for grid search
grid.fit(x_train_tf, y_train)
preds_val = grid.best_estimator_.predict(x_val_tf)
score = f1_score(y_val, preds_val)
score

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.678 total time=   0.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.683 total time=   0.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.692 total time=   0.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.686 total time=   0.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.680 total time=   0.2s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.859 total time=   0.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.898 total time=   0.1s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.843 total time=   0.1s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.868 total time=   0.1s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.855 total time=   0.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.920 total time=   0.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

0.8969072164948454

In [30]:
best_params = grid.best_params_
best_params

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [28]:
best_params = grid.best_params_
model = SVC(         
        **best_params
    )
pipeline = Pipeline(steps=[('sampling', oversample),
                    ('classifier', model)])

pipeline.fit(x_train_tf, y_train)
preds_test = model.predict(x_test_tf)
f1score = f1_score(y_test, preds_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_test))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       225
           1       0.91      0.94      0.93       113

    accuracy                           0.95       338
   macro avg       0.94      0.95      0.94       338
weighted avg       0.95      0.95      0.95       338

